In [1]:
from pyspark.sql import SparkSession

In [8]:
import numpy as np

In [3]:
# # Tạo phiên Spark và ngữ cảnh
# spark = (SparkSession
#          .builder
#          .appName('kafka-streaming-consumer')
#          .config("spark.master", "local[*]")  # Sử dụng tất cả các core trong Docker
#          .getOrCreate())

In [2]:
# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('kafka-streaming')
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
         .getOrCreate())
sc = spark.sparkContext

In [3]:
# Khởi tạo Kafka consumer
kafka_bootstrap_servers = 'kafka1:19091,kafka2:19092,kafka3:19093'  # Địa chỉ Kafka của các container Kafka
kafka_topic = 'trading'  # Tên topic Kafka
# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
  .option("subscribe", kafka_topic) \
  .load()
df.selectExpr("CAST(value AS STRING)")

DataFrame[value: string]

In [12]:
# Hiển thị dữ liệu ra màn hình
query = (df.writeStream
         .outputMode("append")
         .format("console")
         .trigger(processingTime='5 seconds')
         .start())

query.awaitTermination()

KeyboardInterrupt: 

In [11]:
# # Chuyển đổi dữ liệu thành mảng numpy và in ra
# def process_row(row):
#     data = np.frombuffer(row.value.encode(), dtype=np.float32)
#     print(data)
    
# # Áp dụng hàm process_row cho mỗi bản ghi
# query = df.writeStream \
#     .outputMode("append") \
#     .foreach(process_row) \
#     .start()

# query.awaitTermination()

In [17]:
# writer = df.writeStream \
#     .trigger(processingTime='5 seconds') \
#     .outputMode('append') \
#     .format('memory') \
#     .queryName('myQuery') \
#     .start()

# while True:
#     spark.sql('SELECT COUNT(*) FROM myQuery').show()  # Kiểm tra số lượng dòng dữ liệu đã tích lũy trong batch
#     if spark.sql('SELECT COUNT(*) FROM myQuery').first()[0] >= 5:
#         # Đưa dữ liệu vào mô hình LSTM
#         # ...
#         # Reset dữ liệu đã xử lý trong batch
#         spark.sql('DELETE FROM myQuery')


In [11]:
query.toDF()

AttributeError: 'StreamingQuery' object has no attribute 'toDF'